# Assignment 5
Author: Anna Lozenko

In this Jupyter notebook, I will analyze Ireland's population data from a CSV file focusing on the differences between sexes by age groups.

The CSV file has been downloaded from the [Central Statistics Office Ireland](https://data.cso.ie/#) website, for census year 2022.

In [1]:
import pandas as pd

## Part 1: Data Cleaning

In [2]:
# import the population data from the CSV file as a Pandas DataFrame and display the first 5 rows
df = pd.read_csv("population.csv")
print(df.head(5))

  Statistic Label  CensusYear         Sex Single Year of Age  \
0      Population        2022  Both sexes           All ages   
1      Population        2022  Both sexes           All ages   
2      Population        2022  Both sexes           All ages   
3      Population        2022  Both sexes           All ages   
4      Population        2022  Both sexes           All ages   

                 Administrative Counties    UNIT    VALUE  
0                                Ireland  Number  5149139  
1                  Carlow County Council  Number    61968  
2                    Dublin City Council  Number   592713  
3  Dún Laoghaire Rathdown County Council  Number   233860  
4                  Fingal County Council  Number   330506  


Drop unnecessary row values from the dataframe, as we are interested in Ireland only, and specific age groups and sexes.

In [3]:
# drop all counties except 'Ireland'
df = df[df['Administrative Counties'] != 'Ireland']
#drop all rows with 'All ages' in the 'Single Year of Age' column
df = df[df['Single Year of Age'] != 'All ages']
#drop all rows with 'Both sexes' in the 'Sex' column
df = df[df['Sex'] != 'Both sexes']

Drop unnecessary columns.

In [4]:
# Get the list of column names
df.columns.values.tolist()

['Statistic Label',
 'CensusYear',
 'Sex',
 'Single Year of Age',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [5]:
#drop unnecessary columns
df = df[['Sex', 'Single Year of Age', 'VALUE']]
df.head(5)

,Sex,Single Year of Age,VALUE
3297,Male,Under 1 year,346
3298,Male,Under 1 year,3188
3299,Male,Under 1 year,1269
3300,Male,Under 1 year,2059
3301,Male,Under 1 year,1855


Improve data types consistency in the 'Single Year of Age' column by replacing non-numeric values with numeric ones and converting the column to integer type.

In [6]:
# replace "Under 1 year" with "0"
df["Single Year of Age"] = df["Single Year of Age"].str.replace("Under 1 year", "0", regex=True)
df.head()

,Sex,Single Year of Age,VALUE
3297,Male,0,346
3298,Male,0,3188
3299,Male,0,1269
3300,Male,0,2059
3301,Male,0,1855


In [7]:
# replace "100 years and over" with "100"
df["Single Year of Age"] = df["Single Year of Age"].str.replace("100 years and over", "100", regex=True)
df.tail()

,Sex,Single Year of Age,VALUE
9787,Female,100,7
9788,Female,100,9
9789,Female,100,12
9790,Female,100,31
9791,Female,100,7


In [8]:
# remove any remaining non-numeric characters (like "years") from the 'Single Year of Age' column and convert the column to integer type
df["Single Year of Age"] = df["Single Year of Age"].str.replace('\\D', '', regex=True).astype(int)
#check the data types of the columns
df.dtypes

Sex                   object
Single Year of Age     int32
VALUE                  int64
dtype: object

Create a pivot table with clean data and save it to a new CSV file. The pivot table should have 'Single Year of Age' as the index, 'Sex' as the columns, and 'VALUE' as the values.

In [9]:
# create the pivot table and display the first 5 rows
ready_df = pd.pivot_table(df, "VALUE", "Single Year of Age", "Sex")
ready_df.head()

Sex,Female,Male
Single Year of Age,,
0,909.225806,955.161290
1,888.548387,931.451613
2,934.645161,975.354839
3,951.064516,1000.032258
4,961.903226,1022.129032


In [10]:
#save the pivot table to a new CSV file
ready_df.to_csv("cleaned_population.csv")

## Part 2: Data Analysis

Analyze the cleaned data by calculating the weighted mean age by sex.

In [11]:
ready_df.describe()

Sex,Female,Male
count,101.000000,101.000000
mean,831.871607,812.695305
std,399.652916,411.360302
min,10.838710,3.387097
25%,590.129032,563.645161
50%,961.903226,975.290323
75%,1112.548387,1126.645161
max,1409.548387,1343.354839


In [17]:
#check column names
df.columns.values.tolist()

['Sex', 'Single Year of Age', 'VALUE']

In [18]:
# check that age values are in the index
ready_df.index

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100],
      dtype='int32', name='Single Year of Age', length=101)

In [19]:
# calculate the weighted mean age by sex for females
wm_fem = (ready_df.index * ready_df["Female"]).sum() / ready_df["Female"].sum()
print("Weighted mean age for females:", wm_fem)

Weighted mean age for females: 38.93979589877869


In [20]:
# calculate the weighted mean age by sex for males
wm_male = (ready_df.index * ready_df["Male"]).sum() / ready_df["Male"].sum()
print("Weighted mean age for males:", wm_male)

Weighted mean age for males: 37.73944773710391


Calculate the differences between the sexes by age, i.e compute the difference between the number of males and females at each age. The result will be displayed as a new column in the pivot table.

In [21]:
#create the new column 'Difference' in the pivot table
ready_df["Difference"] = ready_df["Male"] - ready_df["Female"]
ready_df.head()


Sex,Female,Male,Difference
Single Year of Age,,,
0,909.225806,955.161290,45.935484
1,888.548387,931.451613,42.903226
2,934.645161,975.354839,40.709677
3,951.064516,1000.032258,48.967742
4,961.903226,1022.129032,60.225806


Positive values in the 'Difference' column indicate that there are more males than females at that age, while negative values indicate that there are more females.